
# TradingView Strategy Backtest Playground

Notebook ini memuat pipeline lengkap untuk menguji sinyal strategi yang diekspor dari TradingView. Ia memandu mulai dari memuat data CSV, mengadaptasikannya ke format QF-Lib, menjalankan backtest, hingga analisis trade kalah dan eksperimen optimasi parameter tambahan.



> **Struktur notebook**
>
> 1. Parameter input & pemuatan data
> 2. Adaptasi data ke QF-Lib
> 3. Strategi berbasis sinyal
> 4. Menjalankan backtest & mengekstrak hasil
> 5. Visualisasi
> 6. Analisis trade kalah & investigasi
> 7. Eksplor optimasi parameter
> 8. Dokumentasi & reusable structure


In [2]:

from __future__ import annotations

import sys
from pathlib import Path
from typing import Dict, Iterable

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8-darkgrid")

PROJECT_ROOT = Path('..').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))
if str(PROJECT_ROOT / 'src') not in sys.path:
    sys.path.append(str(PROJECT_ROOT / 'src'))

from src.strategy_backtest import (
    SignalBacktester,
    get_strategy,
    list_strategies,
    load_strategy_csv,
)


## 1. Parameter input & pemuatan data

In [3]:

DATA_FILE = 'data/OKX_BTCUSDT, 1D.csv'
TIME_COLUMN = 'time'
PRICE_COLUMN = 'close'
ASSET_SYMBOL = 'BTCUSDT'

STRATEGY_NAME = 'ema112_atr'
STRATEGY_KWARGS = {
    'fast_span': 50,
    'slow_span': 112,
    'atr_window': 14,
    'atr_multiplier': 1.5,
}

OPTIMISATION_GRID = {
    'atr_multiplier': [1.0, 1.5, 2.0],
}


In [4]:

data, column_mapping = load_strategy_csv(DATA_FILE, time_column=TIME_COLUMN)
print(f'Dataset berisi {len(data):,} bar dengan {len(data.columns)} kolom.')
print('Contoh mapping kolom (sanitised -> original):')
for alias, original in list(column_mapping.items())[:10]:
    print(f'  {alias} -> {original}')

data.head()


FileNotFoundError: CSV file not found: data\OKX_BTCUSDT, 1D.csv


## 2. Adaptasi data ke QF-Lib

Data sudah menggunakan `DatetimeIndex` dan kolom harga numerik sehingga siap dipakai sebagai sumber harga untuk QF-Lib. Keluaran sanitasi kolom memudahkan pemanggilan strategi Python di langkah berikutnya.



## 3. Strategi berbasis sinyal

Strategi yang dipakai saat ini adalah **EMA 112 dengan exit ATR**:

- **Entry**: ketika EMA cepat (default 50) menyilang ke atas EMA lambat (default 112).
- **Exit**: ketika harga penutupan turun di bawah garis trailing stop, yaitu EMA lambat dikurangi ATR * multiplier.
- **Parameter**: periode EMA dan konfigurasi ATR dapat diubah dari sel parameter di atas.

Strategi tambahan dapat ditaruh dalam folder `src/strategy_backtest/strategies/` dan otomatis terdaftar menggunakan `registry`.


In [ ]:

available = list_strategies()
print('Strategi tersedia:', ', '.join(available))
strategy = get_strategy(STRATEGY_NAME, **STRATEGY_KWARGS)
print('
Deskripsi strategi:')
print(f"- Nama: {strategy.metadata.name}")
print(f"- Deskripsi: {strategy.metadata.description}")
print(f"- Entry: {strategy.metadata.entry}")
print(f"- Exit: {strategy.metadata.exit}")
print(f"- Parameter default: {strategy.metadata.parameters}")


In [ ]:

signals = strategy.generate_signals(data)
print('Kolom sinyal:', list(signals.columns))
signals.head()


## 4. Menjalankan backtest & mengekstrak hasil

In [ ]:

backtester = SignalBacktester(data=data, price_column=PRICE_COLUMN)
outputs = backtester.run(signals)

metrics = outputs.metrics
trade_summary = outputs.trade_summary
print('Metrik performa (QF-Lib / fallback):')
for key, value in metrics.items():
    if isinstance(value, (int, float, np.floating)):
        print(f'- {key}: {value:.4f}')
    else:
        print(f'- {key}: {value}')

print('
Ringkasan trade:')
for key, value in trade_summary.items():
    print(f'- {key}: {value}')


In [ ]:
outputs.trades.head()

## 5. Visualisasi

In [ ]:

fig, ax = plt.subplots(figsize=(14, 6))
close_prices = data[PRICE_COLUMN]
ax.plot(close_prices.index, close_prices, label='Close', color='C0')

if 'ema_fast' in signals.columns:
    ax.plot(signals.index, signals['ema_fast'], label='EMA Fast', color='C1', linewidth=1.2)
if 'ema_slow' in signals.columns:
    ax.plot(signals.index, signals['ema_slow'], label='EMA Slow', color='C2', linewidth=1.2)
if 'atr_trailing_stop' in signals.columns:
    ax.plot(signals.index, signals['atr_trailing_stop'], label='ATR Stop', color='C3', linestyle='--')

long_entries = signals['long_entry'].fillna(False)
long_exits = signals['long_exit'].fillna(False)
ax.scatter(close_prices.index[long_entries], close_prices[long_entries], marker='^', color='green', label='Long Entry')
ax.scatter(close_prices.index[long_exits], close_prices[long_exits], marker='v', color='red', label='Long Exit')

ax.set_title(f'{ASSET_SYMBOL} Close dengan Sinyal {STRATEGY_NAME}')
ax.set_ylabel('Harga')
ax.legend()
plt.show()


In [ ]:

fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(outputs.results.index, outputs.results['equity_curve'], color='C4', label='Equity Curve')
ax.set_title('Equity Curve Strategi')
ax.set_ylabel('Notional')
ax.legend()
plt.show()

fig, ax = plt.subplots(figsize=(10, 4))
ax.hist(outputs.trades['pnl_pct'], bins=20, color='C0', alpha=0.7)
ax.set_title('Distribusi PnL per Trade')
ax.set_xlabel('PnL %')
ax.set_ylabel('Frekuensi')
plt.show()


## 6. Analisis trade kalah & investigasi

In [ ]:

losing_trades = outputs.trades[outputs.trades['pnl_pct'] < 0]
print(f'Jumlah trade kalah: {len(losing_trades)} dari total {len(outputs.trades)}')
if not losing_trades.empty:
    print('
Statistik ringkas trade kalah:')
    display(losing_trades[['pnl_pct', 'bars_held', 'exit_reason']].describe(include='all'))

    grouped = losing_trades.groupby('exit_reason')['pnl_pct'].agg(['count', 'mean', 'min'])
    print('
Kinerja berdasarkan alasan exit:')
    display(grouped)

    context_cols = [col for col in outputs.trades.columns if col.startswith('entry_') or col.startswith('exit_')]
    if context_cols:
        print('
Contoh konteks indikator pada trade kalah:')
        display(losing_trades[['trade_id', 'direction'] + context_cols].head())
else:
    print('Tidak ada trade kalah pada konfigurasi ini.')


## 7. Eksplor optimasi parameter

In [ ]:

def sweep_parameters(base_kwargs: Dict[str, float], grid: Dict[str, Iterable[float]]):
    if not grid:
        return []
    results = []
    for multiplier in grid.get('atr_multiplier', []):
        kwargs = dict(base_kwargs)
        kwargs['atr_multiplier'] = multiplier
        strat = get_strategy(STRATEGY_NAME, **kwargs)
        sig = strat.generate_signals(data)
        backtest = backtester.run(sig)
        results.append({
            'atr_multiplier': multiplier,
            'total_return': backtest.metrics.get('total_return', np.nan),
            'sharpe_ratio': backtest.metrics.get('sharpe_ratio', np.nan),
            'trades': backtest.trade_summary['total_trades'],
        })
    return results

opt_results = sweep_parameters(STRATEGY_KWARGS, OPTIMISATION_GRID)
if opt_results:
    optimisation_df = pd.DataFrame(opt_results).sort_values('sharpe_ratio', ascending=False)
    display(optimisation_df)
else:
    print('Grid optimasi kosong – lewati tahap ini.')



## 8. Dokumentasi & reusable structure

- Ganti `DATA_FILE` untuk menguji dataset TradingView lain yang memakai format serupa.
- Tambah atau modifikasi strategi di `src/strategy_backtest/strategies/`, lalu daftarkan nama modul di `registry.py`.
- Notebook ini mengandalkan `SignalBacktester` untuk menghasilkan trade log, metrik QF-Lib, serta visualisasi standar sehingga langkah analisis tetap konsisten antar strategi.
- Simpan variasi eksperimen (mis. hasil optimasi) ke file CSV tambahan bila diperlukan untuk laporan lebih lanjut.
